In [24]:
#import datetime
#import time
#ts = time.time()
#Time_Initiation = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')

In [1]:
#### Convert PDF into JPGs ####

#Import the packages
import wand 
from wand.image import Image as wi
import os

#Function for converting the PDF to image 
def pdf_image(pdf_name,filefolder):
    file_destination = os.path.join(filefolder, pdf_name)
    pdf = wi(filename = file_destination , resolution=500)
    pdfimage = pdf.convert("png")
    i=1
    for img in pdfimage.sequence:
        page = wi(image=img)
        page.save(filename= file_destination + str(i)+".png")
        i +=1

In [2]:
#Define the function to add the data to excel sheet
def update_excel_sheet(Consignee_list,file_destination):
#Initialize the variables
    cols = 1
    counter = 0 
    rows = 0 
    wb = op.load_workbook(file_destination)

#Get the last row for the first column 
    rows = len(wb['Master Data']['A']) + 1

#Open the master data sheet
    ws = wb['Master Data']

# Loop for writing the data in excel 
    for counter in range(len(Consignee_list)):
        ws.cell(row = rows,column = cols+counter).value = Consignee_list[counter]
        counter = counter + 1
    
#Save and close the excel 
    wb.save(file_destination)
    wb.close()

In [7]:
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
from pytesseract import Output
import cv2
import xlsxwriter
import openpyxl as op 

#Function to change convert image to text and updating the info in excel. 
def Convert_image_text (image_name,file_destination,filefolder):

#Example of adding any additional options.
    custom_oem_psm_config = r'--oem 3 --psm 6'

# Store the path for tesseract for the executable file.
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' 
    
#Enter the image loaction
    #filefolder
    file_destination1 = os.path.join(filefolder, image_name)
    pic_location = Image.open(file_destination1)
    my_string = pytesseract.image_to_string(pic_location, config=custom_oem_psm_config) 
    Consignee_Address = ""
    Destination = ""

#Title for the Invoice crop
    Invoice_Title_Crop = pic_location.crop((1100,230,4000,400))

#Consignee To: crop 
    Consignee_To_Crop = pic_location.crop((200,1575,1000,2100))
    Consignee_list = []

#Convert to store into variables 
    Invoice_Title = pytesseract.image_to_string(Invoice_Title_Crop, config=custom_oem_psm_config).strip()
    Consignee1 = pytesseract.image_to_string(Consignee_To_Crop, config=custom_oem_psm_config)
    Consignee_list = Consignee1.split('\n')
    for ele in Consignee_list:
        if ele != '' and ele != 'Ship-To:':
            Consignee_Address = ele 
            break 

#Check if there is pincode in the address. If yes, change the destination to second last line of the 
    not_country_check = 0 
    for ee in Consignee_list[-1]:
        if ee == '0' or ee == '1' or ee == '2' or ee == '3' or ee == '4' or ee == '5' or ee == '6' or ee == '7' or ee == '8' or ee == '9' or ee == ','or ee == '.':
            not_country_check = 10 
    if not_country_check != 10:
        Destination = Consignee_list[-1]
    
#Get the OBD Number
    OBD_list = []
    OBD_Number_Loc = pic_location.crop((2600,700,4000,1200))
    OBD_image = pytesseract.image_to_string(OBD_Number_Loc, config=custom_oem_psm_config).strip()
    OBD_list = OBD_image.split('\n')
    OBD_Number = OBD_list[-2][-10:]

#Initiate the variables 
    HTC_Number = "" 
    Country_of_Origin = []
    Total = ""
    Total_str = ''
    n = 0 
    info=[]
    Country_of_Origin_str = ""
    S_Country_of_Origin = ''
    Total_str1 = ''
    Total_str_index = 0 
    total_counter = 0

####Extract the values from Invoice and store in variables ####

#Loop for capturing the values 
    for tex in my_string:
        if tex =='H':
            if my_string[n:n+10] == 'HTC Number':
                HTC_Number = str(my_string[n+11:n+32]).strip()
                HTC_index = str(my_string[n+11:n+32]).find('\n')
                HTC_Number = HTC_Number[0:HTC_index-1]
        elif tex =='C':
            if my_string [n:n+17] == 'Country of Origin':
                Country_of_Origin_str = str(my_string[n+19:n+40])
                index = str(my_string[n+19:n+40]).find('\n')
                Country_of_Origin.append(Country_of_Origin_str[0:index])
        elif tex =='T':
            if my_string[n:n+10] == 'Total, DAT':
                Total_str1 = str(my_string[n+2:n+28]).strip()
                Total_str_index = Total_str1.find('\n')
                Total_str = Total_str1[0:Total_str_index-1]
                for ee in reversed(Total_str):
                    if ee == '0' or ee == '1' or ee == '2' or ee == '3' or ee == '4' or ee == '5' or ee == '6' or ee == '7' or ee == '8' or ee == '9' or ee == ','or ee == '.':
                        if ee == '.':
                            if str(Total).find('.')> 0 and total_counter < 11: 
                                Total = Total
                                total_counter = total_counter + 1
                            elif total_counter < 11:
                                Total = Total + str(ee)
                                total_counter = total_counter + 1
                        elif total_counter < 11:
                            Total = Total + str(ee)
                            total_counter = total_counter + 1
                    elif ee == ' ':
                        break 
                
        n = n + 1 
        
    S_Country_of_Origin = ','.join(Country_of_Origin)
    
    info = [Consignee_Address,Destination,Invoice_Title,Total[::-1],OBD_Number,S_Country_of_Origin,HTC_Number] 
    update_excel_sheet(info,file_destination)

In [9]:
#Update the progress and display the summary
#print('Completed!')
#print('Time Initiated', Time_Initiation)
#ts1 = time.time()
#Time_Ended = datetime.datetime.fromtimestamp(ts1).strftime('%H:%M:%S')
#print('Time Ended', Time_Ended)
#print('Converted PDFs ',pdf_count)

# Pick up the images in the folder
import ipywidgets as widgets
import os
button = widgets.Button(description="Convert Documents.")
output = widgets.Output()
filefolder = input('Please enter the file location.')
display(button, output)

def on_button_clicked(filefolder):
    with output: 
        
        # Create the excel in the required format. 
        import xlsxwriter
        #Define the filename and sheet name 
        filename = 'DA Data Extraction.xlsx'
        sheet_name ='Master Data'
        file_destination = os.path.join(filefolder, filename)

        #Create a new workbook 
        workbook = xlsxwriter.Workbook(file_destination)
        worksheet = workbook.add_worksheet(sheet_name)

        #Save the workbook 
        workbook.close()

        import openpyxl as op 
        from openpyxl import load_workbook

        #Define the function to make the add columns to the master sheet
        wb = op.load_workbook(file_destination)
        master_sheet = wb['Master Data']

        #Enter the column names in the excel sheet
        master_sheet['A1'].value = "Consignee Address"
        master_sheet['B1'].value = "Destination"
        master_sheet['C1'].value = "Invoice Title"
        master_sheet['D1'].value = "Invoice Value"
        master_sheet['E1'].value = "OBD Number"
        master_sheet['F1'].value = "Country of Origin"
        master_sheet['G1'].value = "HTC Number"

        #Save all the changes made to the workbook 
        wb.save(file_destination)
    
        #loop for inputting the file 
        arr = os.listdir(filefolder)
        pdf_count = 0 
        pdf_name =''
        for ee in arr:
            if str(ee).find('.pdf')>0:
                pdf_name = str(ee)
                pdf_image(pdf_name,filefolder) #Call the function for converting the PDF to image 
                pdf_count = pdf_count + 1 
        
        arr = os.listdir(filefolder)
        image_name = ''
        for ee in arr:
            if str(ee).find('.png')>0:
                image_name = str(ee)                              #Identify all the images with extension type "png".  
                Convert_image_text (image_name,file_destination,filefolder)  #Call the function to convert the image data to text and udpate the same in excel. 
                image_name_1 = os.path.join(filefolder, image_name)
                os.remove(image_name_1)                             #Delete the used image file. 
        print("Operation Finished!")

button.on_click(on_button_clicked(filefolder))

Please enter the file location. C:\Python\Scripts\Input


Button(description='Convert Documents.', style=ButtonStyle())

Output()

In [10]:
import ipyvuetify as v
v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['Documents Generation']),
    v.Tab(children=['AWB Generation']),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True, children=[
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                button 
            ]),
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
            
            ]),
            v.Flex(xs12=True, xl4=True, children=[
            
            ]),
        ])
    ]),
    v.TabItem(children=[
        v.Container(children=['Lorum ipsum'])
    ])
])

Tabs(children=[Tab(children=['Documents Generation']), Tab(children=['AWB Generation']), TabItem(children=[Lay…